<center>
<img src="./images/ods_stickers.jpg" />
    
# Introducción al Machine Learning 2020

Basado en material de Pedro Pury y  Sebastian Raschka (sraschka@wisc.edu) Traducido y editado al español por [Ana Georgina Flesia](https://www.linkedin.com/in/georginaflesia/). Este material esta sujeto a los términos y condiciones de la licencia  [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Se permite el uso irrestricto para todo propósito no comercial.

## 1) Métricas de Evaluación



Un clasificador se entrena para discriminar los ejemplos en tres
clases (A, B y C) y en la evaluación sobre el conjunto de test
se obtiene la siguiente matriz de confusión:

        ------------------------------
                        clasificado
                       A    B     C    
        ------------------------------
                  A |  9    3     1
    etiquetado    B |  4    8     2
                  C |  2    1     6






a) Calcular la proporción de aciertos obtenidos.


De la matriz de confusión obtenemos los aciertos de la diagonal, por lo tanto tenemos:

$\hat{p} = \dfrac{9 + 8 + 6}{36} = \dfrac{23}{36} = 0.6389$

b)  Calcular un intervalo de confianza del $95\%$ para
la métrica de exactitud.
Justificar si es válida la aproximación normal para el cálculo.

Tenemos que:

* $n = 36$
* $\hat{p} = 0.6389$
* $1 - \hat{p} = \hat{q}= 0.3611$
* $n\hat{p} = 36 \cdot 0.6389 =23$
* $n\hat{q} = 36 \cdot 0.3611 =13$
* $z_{\alpha/2} = 1.96$

Como se da que $n\hat{p} = 23 > 10$ y $n\hat{q} = 13 > 10$ podemos usar la aproximación normal de modo que:


$ IC(95\%) = \dfrac{\hat{p}+ \dfrac{z_{\alpha/2}^{2}}{2n} \pm \sqrt{\hat{p}(1-\hat{p})\dfrac{z_{\alpha/2}^{2}}{n} + \dfrac{z_{\alpha/2}^{4}}{4n^{2}}}}{1 + \dfrac{z_{\alpha/2}^{2}}{n}} = 0.6252 \pm 0.1498$

In [1]:
import numpy as np 

p_hat = 0.6386 
n = 36
q_hat = 1 - p_hat
z_alpha = 1.96

sq = np.sqrt(p_hat*q_hat*((z_alpha**2)/n) + (z_alpha**4)/(4*(n**2)))
sq = sq / (1 + (z_alpha**2)/n)

m = p_hat + (z_alpha**2)/(2*n)
m = m / (1 + (z_alpha**2)/n)
print(round(m, 4))
print(round(sq, 4))

0.6252
0.1498


c)  Calcular el coeficiente $\kappa$ de Cohen.
¿Qué grado de confiabilidad tiene el clasificador?

$\kappa = \dfrac{p_{0} - p_{e}}{1 - p_{e}}$

donde $p_{0}:$ proporción de acuerdo observado y $p_{e}:$ proporción de acuerdo independiente

De lo anterior nos queda que: $p_{0} = \hat{p} = 0.6389$

Para calcular $p_{e}$:

* $p_{Etiq}(A) = 13/36$
* $p_{Etiq}(B) = 14/36$
* $p_{Etiq}(C) = 9/36$
* $p_{Clas}(A) = 15/36$
* $p_{Clas}(B) = 12/36$
* $p_{Clas}(C) = 9/36$

$p_{e} = p_{Etiq}(A)p_{Clas}(A) + p_{Etiq}(B)p_{Clas}(B) + p_{Etiq}(C)p_{Clas}(C) = \dfrac{13 \cdot 15 + 14 \cdot 12 + 9 \cdot 9}{36^{2}} = 0.3426$ 

Y queda que:

$\kappa = \dfrac{p_{0} - p_{e}}{1 - p_{e}} = \dfrac{0.6389 - 0.3426}{1 - 0.3426} = 0.4507$

El clasificador tiene confiabilidad **moderada** en el acuerdo.

## 2) Bagging

En esta segunda parte de esta tarea, combinará múltiples árboles de decisión dentro de un clasificadorde Bagging. Esta vez, utilizaremos el algoritmo del árbol de decisión `DecisionTreeClassifier` implementado en scikit-learn (que es una variante del algoritmo CART para divisiones binarias).


### 2.1 Bootsrapping

Bagging depende del muestreo bootstrap. Entonces, como primer paso, su tarea es implementar una función para generar muestras de arranque. En este ejercicio, por simplicidad, realizaremos los cálculos basados en el conjunto de datos de Iris.



In [2]:
# DO NOT EDIT OR DELETE THIS CELL
import numpy as np
from mlxtend.data import iris_data
X, y = iris_data()

print('Number of examples:', X.shape[0])
print('Number of features:', X.shape[1])
print('Unique class labels:', np.unique(y))

Number of examples: 150
Number of features: 4
Unique class labels: [0 1 2]


Usaremos la función de scikit-learn `train_test_split` 
función para dividir el conjunto de datos en un conjunto de entrenamiento y prueba.


- El conjunto de prueba debe contener 45 ejemplos, y el conjunto de entrenamiento debe contener 105 ejemplos.
- Para garantizar resultados reproducibles, utilice '123' como semilla aleatoria.
- Realice una división estratificada.

In [3]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=45, 
                                                    random_state=123,
                                                    stratify=y)

print('Number of training examples:', X_train.shape[0])
print('Number of test examples:', X_test.shape[0])

Number of training examples: 105
Number of test examples: 45


Agregué la siguiente celda de código para su conveniencia para verificar su solución. Si sus resultados no coinciden con los resultados que se muestran a continuación, hay un error en su implementación.

In [4]:
# DO NOT EDIT OR DELETE THIS CELL

print('Number of training examples:', X_train.shape[0])
print('Number of test examples:', X_test.shape[0])


#Number of training examples: 105
#Number of test examples: 45


Number of training examples: 105
Number of test examples: 45


A continuación, estamos implementando una función para generar muestras bootstrap del conjunto de entrenamiento. En particular, realizaremos el bootstrapping de la siguiente manera:

- Cree una matriz de índice con valores 0, ..., 104.
- Extraiga una muestra aleatoria (con reemplazo) de esta matriz de índices utilizando el método  `choice` del objeto `RandomState` de  NumPy  que se pasa a la función como rng.
- Seleccione ejemplos de entrenamiento de la matriz X y etiquetas del vector y utilizando la nueva muestra de índices.

In [5]:
def draw_bootstrap_sample(rng, X, y):
    sample_indices = [i for i in range(0, 105)]
    bootstrap_indices = rng.choice(sample_indices, replace=True,
                                   size=105)
    return X[bootstrap_indices], y[bootstrap_indices]


Agregué la siguiente celda de código para su conveniencia para verificar su solución. Si sus resultados no coinciden con los resultados que se muestran a continuación, hay un error en su implementación de la función `draw_bootstrap_sample`.

In [6]:
# DO NOT EDIT OR DELETE THIS CELL

rng = np.random.RandomState(123)
X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)

print('Number of training inputs from bootstrap round:', X_boot.shape[0])
print('Number of training labels from bootstrap round:', y_boot.shape[0])
print('Labels:\n', y_boot)

#Number of training inputs from bootstrap round: 105
#Number of training labels from bootstrap round: 105
#Labels:
# [0 0 1 0 0 1 2 0 2 1 0 0 2 1 1 1 1 2 1 1 2 0 2 1 2 1 1 1 0 1 0 0 1 2 0 0 0
# 0 2 1 1 2 1 2 1 1 2 1 2 0 1 1 2 2 1 0 1 0 2 2 0 1 0 2 0 0 0 0 1 2 0 0 1 0
# 1 1 0 1 1 2 2 0 2 0 2 0 1 1 2 2 0 2 2 2 0 1 0 1 2 2 2 1 0 0 0]

Number of training inputs from bootstrap round: 105
Number of training labels from bootstrap round: 105
Labels:
 [0 0 1 0 0 1 2 0 2 1 0 0 2 1 1 1 1 2 1 1 2 0 2 1 2 1 1 1 0 1 0 0 1 2 0 0 0
 0 2 1 1 2 1 2 1 1 2 1 2 0 1 1 2 2 1 0 1 0 2 2 0 1 0 2 0 0 0 0 1 2 0 0 1 0
 1 1 0 1 1 2 2 0 2 0 2 0 1 1 2 2 0 2 2 2 0 1 0 1 2 2 2 1 0 0 0]


### 2.2 Clasificador Baggging a partir de árboles de decisión. 


En esta sección, Ud implementará un algoritmo de bagging basado en `DecisionTree Classifier`. Como ayuda se proporciona una solución parcial.

In [7]:
from sklearn.tree import DecisionTreeClassifier


class BaggingClassifier(object):
    
    def __init__(self, num_trees=10, random_state=123):
        self.num_trees = num_trees
        self.rng = np.random.RandomState(random_state)
        
    def fit(self, X, y):
        self.trees_ = [DecisionTreeClassifier(random_state=self.rng) for i in range(self.num_trees)]
        for i in range(self.num_trees):
            X_boot, y_boot = draw_bootstrap_sample(self.rng, X, y)
            self.trees_[i].fit(X_boot, y_boot)

    def predict(self, X):
        ary = np.zeros((X.shape[0], len(self.trees_)), dtype=np.int)
        for i in range(len(self.trees_)):
            ary[:, i] = self.trees_[i].predict(X)

        maj = np.apply_along_axis(lambda x:
                                  np.argmax(np.bincount(x)),
                                            axis=1,
                                            arr=ary)
        return maj

Agregué la siguiente celda de código para su conveniencia para verificar su solución. Si sus resultados no coinciden con los resultados que se muestran a continuación, hay un error en su implementación de la función `BaggingClassifier()`.

In [8]:
# DO NOT EDIT OR DELETE THIS CELL

model = BaggingClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print('Individual Tree Accuracies:')
for tree in model.trees_:
    predictions = tree.predict(X_test) 
    print('%.1f%%' % ((predictions == y_test).sum() / X_test.shape[0] * 100))

print('\nBagging Test Accuracy: %.1f%%' % ((predictions == y_test).sum() / X_test.shape[0] * 100))

#Individual Tree Accuracies:
#88.9%
#93.3%
#97.8%
#93.3%
#93.3%
#93.3%
#91.1%
#97.8%
#97.8%
#97.8%

#Bagging Test Accuracy: 97.8%
    

Individual Tree Accuracies:
88.9%
93.3%
97.8%
93.3%
93.3%
93.3%
91.1%
97.8%
97.8%
97.8%

Bagging Test Accuracy: 97.8%


## 3) Descomposición de sesgo-varianza 


En este ejercicio se le pedirá que calcule los componentes de varianza y sesgo de la función de pérdida 0-1 que discutimos en clase.

- En particular, calculará el sesgo promedio y la varianza promedio sobre todos los ejemplos de prueba (en lugar de un solo ejemplo de prueba).

- El conjunto de datos que utilizará como conjunto de entrenamiento y conjunto de prueba es el conjunto de datos de Iris que ya dividió en `X_train` /` y_train` y `X_test` /` y_test` anteriormente.

- Dado que no tenemos conjuntos de datos de entrenamiento ilimitados para estimar los parámetros, utilizaremos bootstrapping para simular "nuevos" conjuntos de entrenamiento.

### 3.1 Descomposición de sesgo-varianza  de la pérdida 0-1 para árboles de decisión



En esta primera parte, calculará los  sesgo y varianza promedios sobre los ejemplos del conjunto de prueba para el algoritmo del árbol de decisión implementado en scikit-learn en los datos de Iris.

Ya implementé el código para calcular la "predicción principal" para usted:

In [9]:
# DO NOT EDIT OR DELETE THIS CELL

rng = np.random.RandomState(123)

num_bootstrap = 200

all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype=np.int)

for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = DecisionTreeClassifier(random_state=66).fit(X_boot, y_boot).predict(X_test)
    print(pred)
    all_pred[i] = pred

    
main_predictions = np.apply_along_axis(lambda x:
                                       np.argmax(np.bincount(x)),
                                       axis=0,
                                       arr=all_pred)

[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 1 2 2 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 1 2 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 2 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 2 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 1 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 1 2 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 1 1 1 1 1
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1

[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 2 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 2 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 2 2 2 2 2 1 1 1 2
 2 2 0 2 1 1 2 1]
[2 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 1 2 1 1 2
 2 2 0 2 1 1 2 1]
[1 2 1 1 2 0 0 1 0 0 2 0 2 0 1 0 2 2 0 0 0 0 0 1 0 1 2 0 1 2 2 2 2 1 1

Tenga en cuenta que `all_pred` es una matriz 2D de dimensión $\mathbb{R}^{m \times n_{test}}$, donde $m$ es el número de iteraciones bootstrap  y $n_{test}$ es el número de ejemplos en el conjunto de prueba. En otras palabras, cada una de las 200 filas de esta matriz almacena las predicciones de una hipótesis de árbol de decisión en particular para los 45 puntos de datos de prueba.

Su primera tarea es calcular el sesgo promedio sobre todos los ejemplos de prueba:

In [10]:
def calculate_bias(true_labels, main_predictions):
    biases = (true_labels != main_predictions).astype(int)
    bias = round(np.count_nonzero(biases)/true_labels.shape[0], 3)
    return bias

bias = calculate_bias(y_test, main_predictions)
print('Average bias:', bias)

Average bias: 0.022


Su segunda tarea es calcular la varianza promedio sobre todos los ejemplos de prueba:

In [11]:
def calculate_variance(all_pred, main_predictions):
    variance = (all_pred != main_predictions).astype(int)
    var = round(np.count_nonzero(variance)/variance.size, 3)
    return var

var = calculate_variance(all_pred, main_predictions)
print('Average variance:', var)

Average variance: 0.035


Sugerencia: Los valores promedio de sesgo y varianza son escalares, no vectores o matrices. En otras palabras, para cada una de las celdas de código anteriores, debe devolver un número real (float).

### 3.2 Descomposición de sesgo-varianza  de la pérdida 0-1 para Bagging 

- Utilice el código de la sección anterior, 3.1, para comparar el algoritmo del árbol de decisión con un BaggingClassifier de scikit-learn.

- Informe tanto el sesgo promedio como la varianza promedio como antes, pero use el `BaggingClassifier` en scikit-learn en lugar del` DecisionTreeClassifier`. Puede usar los valores predeterminados de `BaggingClassifier`.

In [12]:
from sklearn.ensemble import BaggingClassifier

rng = np.random.RandomState(123)

num_bootstrap = 200

all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype=np.int)

for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = BaggingClassifier(n_estimators=100, random_state=66).fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred

    
main_predictions = np.apply_along_axis(lambda x:
                                       np.argmax(np.bincount(x)),
                                       axis=0,
                                       arr=all_pred)


bias = calculate_bias(y_test, main_predictions)
var = calculate_variance(all_pred, main_predictions)

print('Average bias:', bias)
print('Average variance:', var)

Average bias: 0.022
Average variance: 0.027


¿Es la varianza promedio de bagging mayor o menor que la varianza promedio del árbol de decisión en 3.1? ¿Y qué 
hay del sesgo promedio? 

La varianza promedio es menor que la del árbol de decisión, mientras que la el bias promedio permanece igual. La menor varianza indica que el clasificador bagging tiene menor tendencia a sobreajustar los datos que el arbol de decisión.

### 3.3 Descomposición de sesgo-varianza  de la pérdida 0-1 para  AdaBoost 


- Utilice el código de la sección anterior, 3.1, para comparar el algoritmo del árbol de decisión con un AdaBoostClassifier de scikit-learn.

- Informe tanto el sesgo promedio como la varianza promedio como antes, pero use el `AdaboostClassifier` en scikit-learn en lugar del` DecisionTreeClassifier`. Puede usar los valores predeterminados de `AdaboostClassifier`.

In [13]:
from sklearn.ensemble import AdaBoostClassifier

rng = np.random.RandomState(123)

num_bootstrap = 200

all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype=np.int)

for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = AdaBoostClassifier(n_estimators=30,
                              random_state=123).fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred

    
main_predictions = np.apply_along_axis(lambda x:
                                       np.argmax(np.bincount(x)),
                                       axis=0,
                                       arr=all_pred)

bias = calculate_bias(y_test, main_predictions)
var = calculate_variance(all_pred, main_predictions)

print('Average bias:', bias)
print('Average variance:', var)

Average bias: 0.044
Average variance: 0.033


¿Es la varianza promedio de boosting mayor o menor que la varianza promedio del árbol de decisión en 3.1? ¿Y qué hay del sesgo promedio? 

La varianza promedio disminuyó y el sesgo promedio aumentó en comparación con el árbol de decisión del 3.1. Los modelos que se consideran para ser usados en Bootsing deben tener baja varianza y alto sesgo ('weakers learners'), ya que lo que busca es reducir el sesgo. Si se cambian el *base_estimator* de **AdaBoostClassifier** por un arbol de decisión con pocos niveles (5 ó 4, árbol que no sobreajusta) entonces el metodo mantiene el sesgo y la varianza apenas incrementa.

<br>
<br>
<br>
<br>
<br>
<br>

## 4)  Random Forest


En este ejercicio se lepide que  que aplique un `RandomForestClassifier` en un pequeño subconjunto (10%) del conjunto de datos de dígitos manuscritos MNIST (http://yann.lecun.com/exdb/mnist/). Por conveniencia, el siguiente código carga este pequeño subconjunto a través de mlxtend:

In [14]:
from mlxtend.data import mnist_data
X, y = mnist_data()

print('Dimensions: %s x %s' % (X.shape[0], X.shape[1]))
print('1st row', X[0])

Dimensions: 5000 x 784
1st row [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.  51. 159. 253. 159.  50.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  48. 238. 252. 252. 252. 237.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  54.
 227. 253. 252. 239. 233. 252.  57.   6.   0. 


El siguiente código muestra como se mezcla el conjunto de datos y se lo divide en 4500 ejemplos de entrenamiento y 500 ejemplos de prueba, respectivamente.

In [15]:
from mlxtend.preprocessing import shuffle_arrays_unison


X, y = shuffle_arrays_unison((X, y), random_seed=1)
X_train, y_train = X[:4500], y[:4500]
X_test, y_test = X[4500:], y[4500:]


Ahora, su tarea es ajustar un clasificador RandomForest en el conjunto de entrenamiento y evaluar su precisión predictiva en el conjunto de prueba.

In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=123)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
acc = ((predictions == y_test).sum() / X_test.shape[0] * 100)
print('Accuracy %.1f%%' % acc)
    
#Accuracy 93.6%

Accuracy 93.6%




Luego, su tarea es cargar una imagen de un dígito (some_digit.png) de este directorio en una matriz de Python y clasificarla usando el modelo de bosque aleatorio. La imagen some_digit.png se muestra a continuación:

![](./images/some_digit.png)


Nota: Para cargar la imagen, debe instalar la biblioteca de imágenes Python PIL. En realidad, es mas recomendable instalar  Pillow. Ejecute uno de los siguientes dos si aún no ha instalado Pillow.
    
- `conda install Pillow`

- `pip install Pillow`

Nuevamente, preescribí parcialmente el código para usted.

In [17]:
from PIL import Image
import numpy as np

def load_image(file_name):
    img = Image.open(file_name)
    img.load()
    data = np.asarray(img, dtype=np.float)
    return data

x_image = load_image('./images/some_digit.png')

In [18]:
# The data needs to be represented as a vector (1 position for each feature)
x_transf = x_image.reshape(1, -1)[0]

# Also, scikit-learn expects 2D arrays, so we need to add a dimension
x_transf = x_transf[np.newaxis, :]

print('Digit:', model.predict(x_transf)[0])

Digit: 5
